In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from transformers import BertTokenizer
bert_path = '../tokenizer/models--bert-base-chinese/snapshots/8d2a91f91cc38c96bb8b4556ba70c392f8d5ee55'
tokenizer = BertTokenizer.from_pretrained(bert_path)
word_map = tokenizer.get_vocab()
max_len = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model, max_len, num_layers = 6):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = self.create_positinal_encoding(max_len, self.d_model)
        self.te = self.create_positinal_encoding(num_layers, self.d_model)
        self.dropout = nn.Dropout(0.1)
    def create_positinal_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model).to(device)
        for pos in range(max_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        return pe
    def forward(self, embedding, layer_idx):
        if layer_idx == 0:
            embedding = self.embed(embedding) * math.sqrt(self.d_model)
        embedding += self.pe[:, :embedding.size(1)]
        embedding += self.te[:, layer_idx, :].unsqueeze(1).repeat(1, embedding.size(1), 1)
        embedding = self.dropout(embedding)
        return embedding
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model):
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)
    def forward(self, query, key, value, mask):
        query = self.query(query)
        key = self.key(key)        
        value = self.value(value)   
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)   
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)  
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)  
        scores = torch.matmul(query, key.permute(0,1,3,2)) / math.sqrt(query.size(-1))
        scores = scores.masked_fill(mask == 0, -1e9)
        weights = F.softmax(scores, dim = -1)
        weights = self.dropout(weights)
        context = torch.matmul(weights, value)
        context = context.permute(0,2,1,3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)
        interacted = self.concat(context)
        return interacted 
class FeedForward(nn.Module):
    def __init__(self, d_model, middle_dim = 2048):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded
class Transformer(nn.Module):
    def __init__(self, d_model, heads, num_layers, word_map, max_len):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.vocab_size = len(word_map)
        self.embed = Embeddings(self.vocab_size, d_model, num_layers = num_layers)
        self.encoder = EncoderLayer(d_model, heads) 
        self.decoder = DecoderLayer(d_model, heads)
        self.logit = nn.Linear(d_model, self.vocab_size)
    def encode(self, src_embeddings, src_mask):
        for i in range(self.num_layers):
            src_embeddings = self.embed(src_embeddings, i)
            src_embeddings = self.encoder(src_embeddings, src_mask)
        return src_embeddings
    def decode(self, tgt_embeddings, target_mask, src_embeddings, src_mask):
        for i in range(self.num_layers):
            tgt_embeddings = self.embed(tgt_embeddings, i)
            tgt_embeddings = self.decoder(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings
    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out
class AdamWarmup:
    def __init__(self, model_size, warmup_steps, optimizer):
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))
    def step(self):
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        self.lr = lr
        self.optimizer.step()
def evaluate(transformer, question, question_mask, max_len, word_map):
    rev_word_map = {v: k for k, v in word_map.items()}
    transformer.eval()
    start_token = word_map['[CLS]']
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)
    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions, dim = 1)
        next_word = next_word.item()
        if next_word == word_map['[SEP]']:
            break
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1)
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
    sen_idx = [w for w in words if w not in {word_map['[CLS]']}]
    sentence = ' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    return sentence

In [2]:
checkpoint = torch.load('../checkpoint/checkpoint_total.pth.tar')
transformer = checkpoint['transformer']

In [3]:
while(1):
    question = input(">>>User: ") 
    if question == 'quit':
        break
    encoded_question = tokenizer.encode_plus(question,return_tensors="pt")
    question_ids = encoded_question["input_ids"]
    enc_qus = question_ids[0].tolist()
    question = torch.LongTensor(enc_qus).to(device).unsqueeze(0)
    question_mask = (question!=0).to(device).unsqueeze(1).unsqueeze(1) 
    sentence = evaluate(transformer, question, question_mask, int(max_len), word_map)
    print("e: "+sentence)

>>>User:  房事之后很累怎么办


e: 你 好 ， 根 据 你 的 叙 述 ， 考 虑 是 炎 症 引 起 的 ， 一 般 和 局 部 病 原 菌 感 染 有 关 系 的 ， 可 以 口 服 阿 莫 灵 ， 甲 硝 唑 治 疗 试 试 ， 注 意 休 息 ， 不 要 刺 激 性 食 物 ， 多 喝 水 ， 慢 慢 会 改 善 的


>>>User:  肛门裂开了怎么办


e: 你 好 ， 肛 裂 是 肛 门 直 肠 粘 膜 下 静 脉 丛 发 生 扩 张 而 形 成 的 柔 软 静 脉 团 ， 可 以 用 无 花 果 叶 子 煎 水 熏 洗 治 疗 ， 有 一 定 效 果


>>>User:  我男的，和老婆房事完很累


...

>>>User:  胰腺癌会不会死啊


e: 胰 腺 癌 的 治 疗 方 法 有 手 术 治 疗 、 化 疗 、 放 疗 、 中 药 治 疗 、 中 药 治 疗 等 。 建 议 您 到 正 规 的 医 院 做 个 详 细 的 检 查 ， 明 确 诊 断 后 再 对 症 治 疗 。 ， 等 到 胰 腺 癌 病 情 得 到 改 善 的 时 候 ， 患 者 切 忌 盲 目 饮 食 ， 最 好 以 清 淡 食 物 为 主 ， 如 果 选 择 了 手 术 治 疗 ， 术 后 一 定 要 重 视 自 身 的 护 理 工 作 ， 并 且 注 意 补 充 维 生 素 ， 多 吃 清 淡 的 食 物 ， 希 望 患 者 病 情 可 以 得 到 缓 解 。


>>>User:  呼吸困难，走路没力气


e: 你 好 ， 这 种 情 况 应 该 是 有 心 脏 病 或 肺 部 疾 病 引 起 的 ， 可 以 应 用 中 医 六 味 地 黄 丸 、 心 得 安 、 谷 维 素 等 药 物 治 疗 看 。 平 时 注 意 保 持 良 好 的 饮 食 及 卫 生 习 惯 。 避 免 刺 激 性 食 物 。


>>>User:  quit
